In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras import layers
from sklearn import preprocessing

import datetime
import math 

In [2]:
def process_data():
    df_test = pd.read_csv(r'../test_data_clean', sep = ",", engine='python')
    df_train = pd.read_csv(r'../train_data_clean', sep = ",", engine='python')

    test = df_test.drop(df_test.columns[0],axis=1)

    test_targets = test.iloc[:,[13]]
    test_inputs = test.loc[:,test.columns != 'target']

    train = df_train.drop(df_train.columns[0],axis=1)
    train_targets = train.iloc[:,[13]]
    train_inputs = train.loc[:,train.columns != 'target']
    
    np.savez('Spotify_data_train', inputs=train_inputs, targets=train_targets)
    np.savez('Spotify_data_test', inputs=test_inputs, targets=test_targets)

    npz = np.load('Spotify_data_train.npz')
    train_inputs,train_targets = npz['inputs'].astype(np.float),npz['targets'].astype(np.int)

    npz = np.load('Spotify_data_test.npz')
    test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

    test_inputs = preprocessing.scale(test_inputs)
    
    return train_inputs,train_targets,test_inputs,test_targets

train_inputs,train_targets,test_inputs,test_targets = process_data()
total_samples = len(train_inputs)
split = math.ceil(total_samples/5)
k = 5

In [3]:
def crossFold(train_inputs,train_targets,i):
    
    if(i==0):
        validation_inputs = preprocessing.scale(train_inputs[:split])
        validation_targets = train_targets[:split]
        inputs = preprocessing.scale(train_inputs[split:])
        targets = train_targets[split:]
    elif(i == 4):
        validation_inputs = preprocessing.scale(train_inputs[i*split:])
        validation_targets = train_targets[i*split:]
        inputs = preprocessing.scale(train_inputs[:i*split])
        targets = train_targets[:i*split]
    else:
        i1 = train_inputs[:i*split]
        i2 = train_inputs[(i+1)*split:]
        inputs = preprocessing.scale(np.concatenate((i1,i2)))
        
        it1 = train_targets[:i*split]
        it2 = train_targets[(i+1)*split:]
        targets = np.concatenate((it1,it2))
        
        validation_inputs = preprocessing.scale(train_inputs[i*split:(i+1)*split])
        validation_targets = train_targets[i*split:(i+1)*split]
        
    return inputs,targets,validation_inputs,validation_targets

In [4]:
#Setting the parameter values we are trying to test

number_of_layers = [10,17,30]
number_of_node = []
optimizers = ['adam','RMSprop','sgd']
batch_sizes = []
max_epochs = []
regulizer_param = []

HP_NUM_LAYERS = hp.HParam('num_layers', hp.Discrete([2,5]))
HP_NUM_NODES = hp.HParam('num_nodes', hp.Discrete([10,17,30])) 
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(optimizers))
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([10,50])) 
HP_MAX_EPOCHS = hp.HParam('max_epochs', hp.Discrete([5,10,25]))
HP_L2 = hp.HParam('l2 regularizer', hp.RealInterval(0.001,0.01))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning1').as_default():
    hp.hparams_config(
    hparams=[HP_NUM_LAYERS,HP_NUM_NODES,HP_OPTIMIZER,HP_BATCH_SIZE, HP_MAX_EPOCHS,HP_L2], 
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )
   
        
def train_test_model(run_dir,hparams,t_inputs,t_targets,v_inputs,v_targets): 
    
    
    input_size = 17 
    output_size = 1
    
    
    model = tf.keras.Sequential()
    model.add(layers.Dense(hparams[HP_NUM_NODES], activation='relu',input_shape=(input_size,)))
    for i in range(hparams[HP_NUM_LAYERS]-1):
        model.add(layers.Dense(hparams[HP_NUM_NODES],kernel_regularizer=tf.keras.regularizers.l2(hparams[HP_L2]), activation='relu'))
    model.add(layers.Dense(output_size, activation='sigmoid'))
    
    model.compile(optimizer=hparams[HP_OPTIMIZER], loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
    
   
    batch_size = hparams[HP_BATCH_SIZE]
    max_epochs = hparams[HP_MAX_EPOCHS]
    
    callbacksList = [
        tf.keras.callbacks.TensorBoard(run_dir,histogram_freq=1,update_freq=1,embeddings_freq=1),
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=8, mode='max',baseline=None),
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=8, verbose=0, mode='min',baseline=0.6)
    ]
        
    history = model.fit(  t_inputs,t_targets,
                          batch_size=batch_size,
                          epochs=max_epochs, 
                          validation_data=(v_inputs, v_targets),
                          callbacks=callbacksList,
                          verbose = 0) 

    test_loss,test_accuracy = model.evaluate(test_inputs, test_targets)
    
    
    return test_loss, test_accuracy   
    
def run(run_dir,hparams,t_inputs,t_targets,v_inputs,v_targets):
    with tf.summary.create_file_writer(run_dir+"/Kfold"+str(i)).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(run_dir+"/Kfold"+str(i),hparams,t_inputs,t_targets,v_inputs,v_targets)
       
        tf.summary.scalar("loss",accuracy[0],step=session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy[1], step=session_num)

    return accuracy[1]
        

In [5]:
session_num = 0
start = datetime.datetime.now()


for num_layers in HP_NUM_LAYERS.domain.values:
    for num_nodes in HP_NUM_NODES.domain.values:
        for optimizer in HP_OPTIMIZER.domain.values:
            for batch_size in HP_BATCH_SIZE.domain.values:
                for max_epoch in HP_MAX_EPOCHS.domain.values:
                        for l2 in (HP_L2.domain.min_value, HP_L2.domain.max_value):
                            hparams = {
                                HP_NUM_LAYERS: num_layers,
                                HP_NUM_NODES: num_nodes,
                                HP_OPTIMIZER: optimizer,
                                HP_BATCH_SIZE: batch_size, 
                                HP_MAX_EPOCHS: max_epoch,
                                HP_L2: l2,
                            }
                            combined_acc = 0
                            
                            start_time_run = datetime.datetime.now()
                            
                            for i in range(5):
                                start_time_fold = datetime.datetime.now()
                                t_inputs,t_targets,v_inputs,v_targets = crossFold(train_inputs,train_targets,i)
                                run_name = "run-%d" % session_num
                                print('----------- Starting trial: %s' % run_name + '-----------')
                                print({h.name: hparams[h] for h in hparams})
                                print("Fold nr "+str(i+1))
                                
                                accuracy = run('logs/hparam_tuning1/' + run_name, hparams,t_inputs,t_targets,v_inputs,v_targets)
                                combined_acc += accuracy
                                
                        
                                finished_time = datetime.datetime.now()
                                
                                print('Total time elapsed: ', finished_time-start,'\n')
                                print('Time elapsed for fold: ', finished_time-start_time_fold ,'\n')
                            
                            combined_acc = combined_acc/k
                            tf.summary.scalar('Combined accuray:', combined_acc , step=1)
                            print("Combined accuray: ",combined_acc,'\n')
                            
                            print('Time elapsed for run: ',finished_time-start_time_run)
                            
                            
                            session_num += 1

print("----- Session finished -----")


----------- Starting trial: run-0-----------
{'num_layers': 1, 'num_nodes': 5, 'optimizer': 'RMSprop', 'batch_size': 5, 'max_epochs': 5, 'l2 regularizer': 0.001}
Fold nr 1
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
71/71 [==============================] - 0s 1ms/step - loss: 0.5987 - accuracy: 0.6636
Total time elapsed:  0:00:13.692025 

Time elapsed for fold:  0:00:13.690994 

----------- Starting trial: run-0-----------
{'num_layers': 1, 'num_nodes': 5, 'optimizer': 'RMSprop', 'batch_size': 5, 'max_epochs': 5, 'l2 regularizer': 0.001}
Fold nr 2
71/71 [==============================] - 0s 981us/step - loss: 0.5937 - accuracy: 0.6759
Total time elapsed:  0:00:26.936246 

Time elapsed for fold:  0:00:13.241227 

----------- Starting trial: run-0-----------
{'num_layers': 1, 'num_nodes': 5, 'optimizer': 'RMSprop', 'batch_size': 5, 'max_epochs': 5, 'l2 regularizer': 0.001}
Fold nr 3
71/71 [==============================] - 0s 999us/step - loss: 0.5903 - accura

71/71 [==============================] - 0s 1ms/step - loss: 0.5860 - accuracy: 0.6900
Total time elapsed:  0:03:29.982389 

Time elapsed for fold:  0:00:22.978575 

----------- Starting trial: run-2-----------
{'num_layers': 1, 'num_nodes': 5, 'optimizer': 'RMSprop', 'batch_size': 5, 'max_epochs': 10, 'l2 regularizer': 0.001}
Fold nr 4
71/71 [==============================] - 0s 973us/step - loss: 0.5868 - accuracy: 0.6799
Total time elapsed:  0:03:56.686718 

Time elapsed for fold:  0:00:26.699344 

----------- Starting trial: run-2-----------
{'num_layers': 1, 'num_nodes': 5, 'optimizer': 'RMSprop', 'batch_size': 5, 'max_epochs': 10, 'l2 regularizer': 0.001}
Fold nr 5
71/71 [==============================] - 0s 835us/step - loss: 0.5967 - accuracy: 0.6781
Total time elapsed:  0:04:17.688268 

Time elapsed for fold:  0:00:20.997554 

Combined accuray:  0.6815499663352966 

Time elapsed for run:  0:02:02.825591
----------- Starting trial: run-3-----------
{'num_layers': 1, 'num_nodes'

71/71 [==============================] - 0s 897us/step - loss: 0.5885 - accuracy: 0.6821
Total time elapsed:  0:10:56.692673 

Time elapsed for fold:  0:00:35.384146 

----------- Starting trial: run-5-----------
{'num_layers': 1, 'num_nodes': 5, 'optimizer': 'RMSprop', 'batch_size': 5, 'max_epochs': 25, 'l2 regularizer': 0.01}
Fold nr 2
71/71 [==============================] - 0s 1ms/step - loss: 0.5788 - accuracy: 0.6909
Total time elapsed:  0:11:58.043471 

Time elapsed for fold:  0:01:01.349800 

----------- Starting trial: run-5-----------
{'num_layers': 1, 'num_nodes': 5, 'optimizer': 'RMSprop', 'batch_size': 5, 'max_epochs': 25, 'l2 regularizer': 0.01}
Fold nr 3
71/71 [==============================] - 0s 817us/step - loss: 0.5907 - accuracy: 0.6764
Total time elapsed:  0:12:19.028645 

Time elapsed for fold:  0:00:20.984178 

----------- Starting trial: run-5-----------
{'num_layers': 1, 'num_nodes': 5, 'optimizer': 'RMSprop', 'batch_size': 5, 'max_epochs': 25, 'l2 regularizer'

71/71 [==============================] - 0s 957us/step - loss: 0.5738 - accuracy: 0.6997
Total time elapsed:  0:22:09.221893 

Time elapsed for fold:  0:00:56.230374 

Combined accuray:  0.687010133266449 

Time elapsed for run:  0:03:41.196356
----------- Starting trial: run-8-----------
{'num_layers': 1, 'num_nodes': 5, 'optimizer': 'RMSprop', 'batch_size': 10, 'max_epochs': 5, 'l2 regularizer': 0.001}
Fold nr 1
71/71 [==============================] - 0s 737us/step - loss: 0.6040 - accuracy: 0.6513
Total time elapsed:  0:22:16.496083 

Time elapsed for fold:  0:00:07.273190 

----------- Starting trial: run-8-----------
{'num_layers': 1, 'num_nodes': 5, 'optimizer': 'RMSprop', 'batch_size': 10, 'max_epochs': 5, 'l2 regularizer': 0.001}
Fold nr 2
71/71 [==============================] - 0s 960us/step - loss: 0.5972 - accuracy: 0.6684
Total time elapsed:  0:22:24.133377 

Time elapsed for fold:  0:00:07.635300 

----------- Starting trial: run-8-----------
{'num_layers': 1, 'num_nodes

71/71 [==============================] - 0s 915us/step - loss: 0.5917 - accuracy: 0.6786
Total time elapsed:  0:23:59.137198 

Time elapsed for fold:  0:00:10.885757 

----------- Starting trial: run-10-----------
{'num_layers': 1, 'num_nodes': 5, 'optimizer': 'RMSprop', 'batch_size': 10, 'max_epochs': 10, 'l2 regularizer': 0.001}
Fold nr 4
71/71 [==============================] - 0s 903us/step - loss: 0.5945 - accuracy: 0.6733
Total time elapsed:  0:24:15.540663 

Time elapsed for fold:  0:00:16.402516 

----------- Starting trial: run-10-----------
{'num_layers': 1, 'num_nodes': 5, 'optimizer': 'RMSprop', 'batch_size': 10, 'max_epochs': 10, 'l2 regularizer': 0.001}
Fold nr 5
71/71 [==============================] - 0s 866us/step - loss: 0.5908 - accuracy: 0.6733
Total time elapsed:  0:24:28.871961 

Time elapsed for fold:  0:00:13.330366 

Combined accuray:  0.6749449849128724 

Time elapsed for run:  0:01:07.576354
----------- Starting trial: run-11-----------
{'num_layers': 1, 'num

...

71/71 [==============================] - 0s 859us/step - loss: 0.5719 - accuracy: 0.6896
Total time elapsed:  8:31:20.552046 

Time elapsed for fold:  0:00:07.144906 

Combined accuray:  0.6840158343315125 

Time elapsed for run:  0:00:31.083421
----------- Starting trial: run-350-----------
{'num_layers': 1, 'num_nodes': 30, 'optimizer': 'adam', 'batch_size': 100, 'max_epochs': 50, 'l2 regularizer': 0.001}
Fold nr 1
71/71 [==============================] - 0s 905us/step - loss: 0.5714 - accuracy: 0.6918
Total time elapsed:  8:31:28.873077 

Time elapsed for fold:  0:00:08.319081 

----------- Starting trial: run-350-----------
{'num_layers': 1, 'num_nodes': 30, 'optimizer': 'adam', 'batch_size': 100, 'max_epochs': 50, 'l2 regularizer': 0.001}
Fold nr 2
71/71 [==============================] - 0s 815us/step - loss: 0.5796 - accuracy: 0.6843
Total time elapsed:  8:31:33.780965 

Time elapsed for fold:  0:00:04.906889 

----------- Starting trial: run-350-----------
{'num_layers': 1, 'nu

71/71 [==============================] - 0s 949us/step - loss: 0.5832 - accuracy: 0.6799
Total time elapsed:  8:33:40.543825 

Time elapsed for fold:  0:00:14.404600 

----------- Starting trial: run-352-----------
{'num_layers': 1, 'num_nodes': 30, 'optimizer': 'sgd', 'batch_size': 5, 'max_epochs': 5, 'l2 regularizer': 0.001}
Fold nr 5
71/71 [==============================] - 0s 900us/step - loss: 0.5828 - accuracy: 0.6878
Total time elapsed:  8:33:58.257953 

Time elapsed for fold:  0:00:17.712133 

Combined accuray:  0.6826067686080932 

Time elapsed for run:  0:01:13.927307
----------- Starting trial: run-353-----------
{'num_layers': 1, 'num_nodes': 30, 'optimizer': 'sgd', 'batch_size': 5, 'max_epochs': 5, 'l2 regularizer': 0.01}
Fold nr 1
71/71 [==============================] - 0s 983us/step - loss: 0.5799 - accuracy: 0.6860
Total time elapsed:  8:34:12.408243 

Time elapsed for fold:  0:00:14.147300 

----------- Starting trial: run-353-----------
{'num_layers': 1, 'num_nodes':

71/71 [==============================] - 0s 923us/step - loss: 0.5808 - accuracy: 0.6821
Total time elapsed:  8:38:15.833882 

Time elapsed for fold:  0:00:26.040762 

----------- Starting trial: run-355-----------
{'num_layers': 1, 'num_nodes': 30, 'optimizer': 'sgd', 'batch_size': 5, 'max_epochs': 10, 'l2 regularizer': 0.01}
Fold nr 3
71/71 [==============================] - 0s 892us/step - loss: 0.5772 - accuracy: 0.6874
Total time elapsed:  8:38:37.342426 

Time elapsed for fold:  0:00:21.506547 

----------- Starting trial: run-355-----------
{'num_layers': 1, 'num_nodes': 30, 'optimizer': 'sgd', 'batch_size': 5, 'max_epochs': 10, 'l2 regularizer': 0.01}
Fold nr 4
71/71 [==============================] - 0s 885us/step - loss: 0.5733 - accuracy: 0.6856
Total time elapsed:  8:39:03.772616 

Time elapsed for fold:  0:00:26.428193 

----------- Starting trial: run-355-----------
{'num_layers': 1, 'num_nodes': 30, 'optimizer': 'sgd', 'batch_size': 5, 'max_epochs': 10, 'l2 regularizer':

71/71 [==============================] - 0s 716us/step - loss: 0.5698 - accuracy: 0.6997
Total time elapsed:  8:48:54.921645 

Time elapsed for fold:  0:00:37.106587 

Combined accuray:  0.6966094136238098 

Time elapsed for run:  0:04:46.839434
----------- Starting trial: run-358-----------
{'num_layers': 1, 'num_nodes': 30, 'optimizer': 'sgd', 'batch_size': 5, 'max_epochs': 50, 'l2 regularizer': 0.001}
Fold nr 1


KeyboardInterrupt: 